# Import

In [1]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml


# URL Rule

In [2]:
# comp_id = 380725    # 기업 고유 UID
p_num = 1      # 리뷰 페이지 (4 reviews / page)

### query로 CompID 가져오기

In [3]:
Keyword = '삼성전자'

In [4]:
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(Keyword))
# print(search_req.text)

In [5]:
search_soup = BeautifulSoup(search_req.text, 'lxml')
search_soup.text

'\n\n통합검색ㅣ캐치\n\n\n, 메인메뉴 바로가기 본문 바로가기   CATCH 전체메뉴 보기 \xa0 통합검색 채용공고 기업정보 뉴스 캐치TV 캐치클래스 검색 더보기 통합검색  검색하기  로그인 회원가입 채용공고 공고캘린더인재PICK 인재PICK메인제안관리기업 기업분석추천기업분석리포트현직자리뷰기업비교캐치클래스 VOD클래스LIVE클래스커리어 콘자소서/면접 자소서면접최종합격캐치TV 뉴스 캐치카페 진단 진단안내기업적합도조직인성선호직무탐색NCS직업기초능력학교회원 진단맞춤형 서비스 이력서 관리자소서 관리입사제안 관리For U나의 검색개인정보관리 캐치알바 기업회원 홈 이벤트고객센터이용가이드      삼성 TOP10 다른 그룹 보기   삼성전자 삼성생명보험 삼성디스플레이 삼성물산 삼성화재해상보험 삼성SDI 삼성증권 삼성전기 삼성엔지니어링 삼성중공업 삼성SDS 호텔신라 삼성카드 삼성전자판매 세메스 삼성바이오로직스 에스원 삼성전자로지텍 삼성웰스토리 삼성전자서비스 제일기획 STM 삼성바이오에피스 HDC신라면세점 삼성메디슨 멀티캠퍼스 삼우종합건축사사무소 스테코 미라콤아이앤씨 삼성자산운용 삼성선물 삼성글로벌리서치 삼성생명서비스손해사정 삼성화재애니카손해사정 휴먼TSS 삼성화재서비스손해사정 시큐아이 삼성코닝어드밴스드글라스 신라에이치엠 삼성카드고객서비스 에스코어 삼성전자서비스씨에스 삼성화재금융서비스보험대리점 삼성벤처투자 삼성라이온즈 삼성생명금융서비스 서울레이크사이드 삼성에스알에이자산운용 씨브이네트 에스에이치피코퍼레이션 SBTM 제일패션리테일 삼성액티브자산운용 수원삼성축구단 에스디플렉스 하만인터내셔널코리아 SU머티리얼즈 에스원CRM 오픈핸즈 삼성에프엔위탁관리부동산투자회사 삼성헤지자산운용 삼성서울병원 삼성물산패션부문 멀티캠퍼스\n                                              관심  경력무관 계약직 전자상품 강사 (계약직, 광주) 채용 오늘마감 에스원CRM\n                                              관심  신입 

In [6]:
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

[<a href="/Comp/CompSummary/380725">
                 삼성전자
             </a>]

In [7]:
comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

'380725'

In [8]:
URL = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=1&isEmploy=false&jobCode='
print(URL)

https://www.catch.co.kr/api/v1.0/comp/reviewInfo/380725/commentList?currentPage=1&pageSize=5000&isNew=false&employType=1&isEmploy=false&jobCode=


# Json to DataFrame 

In [9]:
# Json 형식의 웹 데이터 가져오기
raw_data = requests.get(URL).json()
print(raw_data)

{'reviewList': [{'idx': 209151, 'CompID': '380725', 'CompName': '삼성전자', 'RegDate': '2023-08-10T08:41:40.253Z', 'CI': '380725.jpg', 'EmployText': '현직', 'Gender2': 2, 'RecomName': '추천', 'EmployType': '정규직', 'NewOld': '신입', 'Answer': None, 'Good': '어학  지원\r\n칼퇴  가능\r\n자울 출퇴근제\r\n식사 제공\r\n통근버스', 'Bad': '수직적 조직문화\r\n선택적 복리후생\r\n불투명한 소통\r\n기타등등', 'UsefulY': 0, 'MyUsefulY': None, 'MyOpinion': None, 'CareerYear': 13, 'CareerYearYN': 'Y', 'Keyword1': None, 'Keyword2': None, 'Keyword3': None, 'Keyword1YN': 'N', 'Keyword2YN': 'N', 'Keyword3YN': 'N', 'JobName': '생산관리/공정관리/품질관리', 'Area': '경북', 'MyStarScore': 4}, {'idx': 209113, 'CompID': '380725', 'CompName': '삼성전자', 'RegDate': '2023-08-09T03:31:28.130Z', 'CI': '380725.jpg', 'EmployText': '전직', 'Gender2': 2, 'RecomName': '추천', 'EmployType': '정규직', 'NewOld': '신입', 'Answer': None, 'Good': '눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.', 'Bad': '높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요', 'UsefulY': 0, 'MyUsefulY': None, 'MyOpinion': None, 'CareerYear': 3, 'CareerYearY

In [10]:
data = raw_data['reviewList']
data[0]

{'idx': 209151,
 'CompID': '380725',
 'CompName': '삼성전자',
 'RegDate': '2023-08-10T08:41:40.253Z',
 'CI': '380725.jpg',
 'EmployText': '현직',
 'Gender2': 2,
 'RecomName': '추천',
 'EmployType': '정규직',
 'NewOld': '신입',
 'Answer': None,
 'Good': '어학  지원\r\n칼퇴  가능\r\n자울 출퇴근제\r\n식사 제공\r\n통근버스',
 'Bad': '수직적 조직문화\r\n선택적 복리후생\r\n불투명한 소통\r\n기타등등',
 'UsefulY': 0,
 'MyUsefulY': None,
 'MyOpinion': None,
 'CareerYear': 13,
 'CareerYearYN': 'Y',
 'Keyword1': None,
 'Keyword2': None,
 'Keyword3': None,
 'Keyword1YN': 'N',
 'Keyword2YN': 'N',
 'Keyword3YN': 'N',
 'JobName': '생산관리/공정관리/품질관리',
 'Area': '경북',
 'MyStarScore': 4}

In [11]:
df = pd.DataFrame(data)
df

,idx,CompID,CompName,RegDate,CI,EmployText,Gender2,RecomName,EmployType,NewOld,...,CareerYearYN,Keyword1,Keyword2,Keyword3,Keyword1YN,Keyword2YN,Keyword3YN,JobName,Area,MyStarScore
0,209151,380725,삼성전자,2023-08-10T08:41:40.253Z,380725.jpg,현직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,생산관리/공정관리/품질관리,경북,4.0
1,209113,380725,삼성전자,2023-08-09T03:31:28.130Z,380725.jpg,전직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,생산/제조/설비/조립,경기,3.0
2,208104,380725,삼성전자,2023-06-26T05:25:22.170Z,380725.jpg,전직,2,추천,정규직,신입,...,N,None,None,None,N,N,N,웹개발,서울,4.6
3,208095,380725,삼성전자,2023-06-26T00:15:57.717Z,380725.jpg,현직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,일반영업,서울,2.6
4,207978,380725,삼성전자,2023-06-21T05:23:01.440Z,380725.jpg,전직,2,비추,정규직,경력,...,Y,None,None,None,N,N,N,기획/전략/경영,경기,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,2020-09-26T11:23:26.790Z,380725.jpg,전직,2,추천,정규직,경력,...,N,None,None,None,N,N,N,판매/캐셔/매장관리,None,4.2
302,120816,380725,삼성전자,2020-09-24T05:02:07.343Z,380725.jpg,현직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,기획/전략/경영,None,4.0
303,120504,380725,삼성전자,2020-09-12T07:34:44.560Z,380725.jpg,전직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,재무/세무/IR,None,4.0
304,120412,380725,삼성전자,2020-09-07T12:49:14.490Z,380725.jpg,전직,2,추천,정규직,경력,...,Y,None,None,None,N,N,N,네트워크/서버/보안,None,3.4


In [12]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'CI', 'EmployText', 'Gender2',
       'RecomName', 'EmployType', 'NewOld', 'Answer', 'Good', 'Bad', 'UsefulY',
       'MyUsefulY', 'MyOpinion', 'CareerYear', 'CareerYearYN', 'Keyword1',
       'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN',
       'JobName', 'Area', 'MyStarScore'],
      dtype='object')

In [13]:
df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 'CareerYearYN',  'MyUsefulY', 
            'MyOpinion', 'CareerYear', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
            axis=1, inplace=True)

df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'RecomName',
       'Good', 'Bad', 'JobName', 'MyStarScore'],
      dtype='object')

In [14]:
# 정규식으로 이스케이프 문자 제거        # Good, Bad 리뷰 항목에만 적용.
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df

,idx,CompID,CompName,RegDate,EmployText,RecomName,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,2023-08-10T08:41:40.253Z,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,2023-08-09T03:31:28.130Z,전직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,2023-06-26T05:25:22.170Z,전직,추천,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,2023-06-26T00:15:57.717Z,현직,추천,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,2023-06-21T05:23:01.440Z,전직,비추,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,2020-09-26T11:23:26.790Z,전직,추천,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,2020-09-24T05:02:07.343Z,현직,추천,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,2020-09-12T07:34:44.560Z,전직,추천,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,2020-09-07T12:49:14.490Z,전직,추천,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


In [15]:
# RegDate 컬럼의 데이터 - 문자열 9번째 까지 출력 (YYYYMMDD 형식의 날짜 표기)
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())
df

,idx,CompID,CompName,RegDate,EmployText,RecomName,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,20230810,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,전직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,전직,추천,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,현직,추천,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,전직,비추,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,전직,추천,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,현직,추천,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,전직,추천,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,전직,추천,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# For 문으로 만들어보기
    - 리뷰 데이터가 있는 페이지까지 반복

In [16]:
rv = []

for p_num in range(1, 52) :
    url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=4&isNew=false&employType=1&isEmploy=false&jobCode='
    rv.append(requests.get(url).json()['reviewList'])


data = list(itertools.chain.from_iterable(rv))
df = pd.DataFrame(data)


df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 'CareerYearYN',  'MyUsefulY', 
            'MyOpinion', 'CareerYear', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
            axis=1, inplace=True)
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())


df

,idx,CompID,CompName,RegDate,EmployText,RecomName,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,20230810,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,전직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,전직,추천,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,현직,추천,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,전직,비추,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
199,148439,380725,삼성전자,20211020,전직,비추,돈을 많이 주고 이직에 최소한 경력으로 도움이 된다 복지는 좋다,돈을 많이 주는만큼 일을 많이 시키고 유연근무제도 안유연하고 야근이 많다,반도체/디스플레이,2.4
200,148244,380725,삼성전자,20211020,현직,추천,"자울출퇴근제이고, 합의된 비율로 재택근무가능하다. 예전에 비해 조직 분위기가 강압적...",제품이 너무 많고 일정이 타이트하여 개발자들과 관련부서(검증 등)에 로드가 심하다....,전기/전자/제어,4.0
201,147914,380725,삼성전자,20211020,현직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 연차를 쓸 수 있다.,가끔 업무강도가 너무 높고 야근이 갑자기 생겨서 불편하다.,사무/총무/법무,4.6
202,145509,380725,삼성전자,20210625,전직,추천,업무문화가 계속해서 변화를 시도해서 좋다. 뭔가 비젼을 제시하고 있어서 좋다,정년을 보장받기는 어렵다. 정리해고 할때는 가차없이 내보내고 노조가 없어서 아무소리...,전기/전자/제어,3.0


# While 문으로 만들기 
    - 그 다음 페이지에 리뷰 데이터 개수가 0 이면 멈춤


In [17]:
# len(requests.get(url).json()['reviewList'])

In [18]:
# Catch 사이트에 검색을 하면, 최상위 검색결과의 comp_ui를 받아오기

comp_name = '삼성전자'
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(comp_name)) # Keyword = comp_name
# print(search_req.text)

search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

'380725'

In [19]:
# 위에서 받아온 comp_id를 가지고 데이터 받아와서 데이터프레임으로 만들기

try : 
    # comp_id = 380725
    p_num = 1

    rv = []


    while True : 

        url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=2000&isNew=false&employType=1&isEmploy=false&jobCode='
        rv.append(requests.get(url).json()['reviewList'])

        p_num += 1

        if len(requests.get(url).json()['reviewList']) == 0 :
            break


    data = list(itertools.chain.from_iterable(rv))      # = 리스트 컴프리헨션
    df = pd.DataFrame(data)


    df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 'CareerYearYN',  'MyUsefulY', 
                'MyOpinion', 'CareerYear', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
                axis=1, inplace=True)
    df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())
    


except :
    print("리뷰가 존재하지 않습니다.")


df


,idx,CompID,CompName,RegDate,EmployText,RecomName,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,20230810,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,전직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,전직,추천,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,현직,추천,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,전직,비추,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,전직,추천,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,현직,추천,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,전직,추천,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,전직,추천,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# 컬럼명 및 데이터 변경

### 컬럼명 변경

In [20]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'RecomName',
       'Good', 'Bad', 'JobName', 'MyStarScore'],
      dtype='object')

In [21]:
df.rename(columns=
            {'idx':'review_uid', 
            'CompID':'comp_uid',
            'CompName':'comp_name', 
            'RegDate':'review_date', 
            'EmployText':'is_office', 
            'Good':'review_pos', 
            'Bad':'review_neg', 
            'MyStarScore':'review_rate', 
            'JobName':'position'}, 
            inplace=True)

df.head(1)

,review_uid,comp_uid,comp_name,review_date,is_office,RecomName,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,20230810,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0


### 전/현직 여부를 Boolean 값으로 변경

In [22]:
df[['is_office']]

,is_office
0,현직
1,전직
2,전직
3,현직
4,전직
...,...
301,전직
302,현직
303,전직
304,전직


In [23]:
df['is_office'] = df['is_office'].apply(lambda x: x.replace('현직', '1').strip() == '1')
df[['is_office']]

,is_office
0,True
1,False
2,False
3,True
4,False
...,...
301,False
302,True
303,False
304,False


### 임의로 넣은 review_senti    <<- 모델 완성 후 교체 및 수정 요망 !!!

In [24]:
# 컬럼명 'RecomName'을 'review_senti'로 변경    <-- Catch에서 지원하는 추천/비추천 여부
df.rename(columns={'RecomName':'review_senti'}, inplace=True)
df.head(1)

,review_uid,comp_uid,comp_name,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,20230810,True,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0


In [25]:
# 추천/비추천 여부의 값을 1 | 0  으로 변환
df['review_senti'] = df['review_senti'].replace(['추천', '비추'], [1, 0])
df

,review_uid,comp_uid,comp_name,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,20230810,True,1,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,False,1,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,False,1,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,True,1,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,False,0,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,False,1,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,True,1,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,False,1,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,False,1,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# DataFrame to CSV file

In [26]:
SAVE_PATH = r'./data/'

In [27]:
df['comp_name'][0]

'삼성전자'

In [28]:
comp = df['comp_name'][0]

In [29]:
# 디렉토리 생성 (이미 존재하면 생성하지 않음)

os.makedirs(SAVE_PATH, exist_ok=True)

# csv 파일로 저장.

file_name = f"{comp}_catch.csv"
save_file_path = os.path.join(SAVE_PATH, file_name)

df.to_csv(save_file_path, index=False, encoding = "utf-8")

In [30]:
pd.read_csv(save_file_path)

,review_uid,comp_uid,comp_name,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,20230810,True,1,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,False,1,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,False,1,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,True,1,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,False,0,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,False,1,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,True,1,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,False,1,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,False,1,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# 함수화

In [31]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml

In [32]:
SAVE_PATH = r'./data/'
# SAVE_PATH = r'/app/data/reviews/'     <--- for Docker

In [33]:
re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))

'[<ahref="/Comp/CompSummary/380725">삼성전자</a>]'

In [34]:
re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]

'삼성전자'

In [35]:
def RV_Catch(comp = str): 
    
    search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
    search_req = requests.get(search_url.format(comp)) # Keyword = comp_name
    # print(search_req.text)

    search_soup = BeautifulSoup(search_req.text, 'lxml')
    # search_soup.text
    search_soup.select('li:nth-child(1) > div.txt > p.name > a')

    comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
    comp_name = re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]




    p_num = 1

    rv = []



    try : 

        while True : 

            url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=1&isEmploy=false&jobCode='
            rv.append(requests.get(url).json()['reviewList'])

            p_num += 1

            if len(requests.get(url).json()['reviewList']) == 0 :
                break


        data = list(itertools.chain.from_iterable(rv))
        df = pd.DataFrame(data)


        df.drop(['idx', 'CompID', 'CompName', 'CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 
                'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 
                'EmployType', 'NewOld', 'CareerYear', 'Area'], 
                axis=1, inplace=True)
        

        df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
        df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())

        df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())

        df['EmployText'] = df['EmployText'].apply(lambda x: x.replace('현직', '1').strip() == '1')

        df['RecomName'] = df['RecomName'].replace(['추천', '비추'], [1, 0])


        df.rename(columns=
           {'RegDate':'review_date', 
           'EmployText':'is_office', 
           'Good':'review_pos', 
           'Bad':'review_neg', 
           'MyStarScore':'review_rate', 
           'JobName':'position',
           'RecomName':'review_senti'       # 임의로 쓰는 값 (나중에 모델 돌려서 나온 결과 값으로 대체 할 것.)
           }, inplace=True)




        # csv 파일로 저장.

        os.makedirs(SAVE_PATH, exist_ok=True)

        file_name = f"{comp_name}_catch.csv"
        save_file_path = os.path.join(SAVE_PATH, file_name)

        df.to_csv(save_file_path, index=False, encoding = "utf-8")
        
        return pd.read_csv(save_file_path)


        
    except : 
        print("리뷰가 존재하지 않습니다.")




In [36]:
RV_Catch('SK하이')

,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,20230722,False,1,어르신들도 알 정도로의 인지도의 대기업. 괜찮은 급여수준.,업무량이 꽤 있는 편이라고 생각합니다. 워라밸 측면에서는 좀 아쉬움,기획/전략/경영,3.2
1,20230620,True,1,"1. 유연근무제 , 해피프라이데이 등 근무 자유로움 2. 상대적으로 높은 연봉 3....",1. 반도체 업계 특성상 사이클을 탐 2. 팀바팀이지만 야근하는 경우도 많음 3. ...,반도체/디스플레이,5.0
2,20230611,True,1,팀바팀 부바부가 심하지만 대체로 제조기술쪽 직군은 삼성전자에 비해 편한편이다. 또 ...,다만 제조기술이 아닌팀 그리고 일부 팀들은 야근이 매우 많은팀들이 있긴하다. 추가로...,생산/제조/설비/조립,3.8
3,20230609,True,1,회사 복지 면이 엄청 좋은 것 같다. 회사가 어려워서 복지 면에서 축소가 되지 않을...,보수적인 상사분들이 있다. (보통 나이가 많으신 분들 중에) 유연근무제이긴 하나 부...,생산/제조/설비/조립,4.2
4,20230608,True,1,급여 수준이 높고 유연 근무제가 시행되어 출퇴근이 자유롭다. 한달에 한번 해피프라이...,서울 경기권에서 지역 마다 다르지만 출퇴근 시간이 오래걸린다. 조직 문화가 다소 경...,생산관리/공정관리/품질관리,4.4
...,...,...,...,...,...,...,...
81,20201118,True,1,"대기업인만큼 복지가 잘 되어있음, 하루 네끼 무료","연봉 인상률이 낮은 점, 회사 주변 인프라가 별로 좋지 않다",반도체/디스플레이,4.4
82,20201118,True,1,반도체가 호황이라 돈을 많이 주지만 일하는 만큼 받는건지는 모르겠음,모든 남초 회사가 그러하듯 여혐 분위기가 기본적으로 깔려있음,반도체/디스플레이,3.8
83,20201117,False,1,복지 좋음. 식사 시설 등 제공 좋습니다. 만족해요 다양한 선택권,위치 안좋음. 매일 셔틀 지옥입니다 안좋아요. 서울에서 셔틀 타고 다녀야함,반도체/디스플레이,4.0
84,20201010,True,1,복지혜택이 많다. 월급이 많다. 성과급이 많다,지리적으로 멀다. 일부 직무는 매우 비선호 된다,반도체/디스플레이,5.0


In [37]:
RV_Catch('화인석재')

리뷰가 존재하지 않습니다.


In [38]:
RV_Catch('삼성전자')

,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,20230810,True,1,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,20230809,False,1,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,20230626,False,1,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,20230626,True,1,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,20230621,False,0,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...
301,20200926,False,1,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,20200924,True,1,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,20200912,False,1,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,20200907,False,1,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


In [39]:
RV_Catch('네이버')

,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,20230623,False,1,익히 알려진 네이버 장점들 그대로 입니다. 일반 기업 대비 자유롭고 일만 열심히 하면 됨,다른 테크 회사들 대비해서는 조금 더 경직된 구조이긴 함. 아무래도 대기업이니,기획/전략/경영,3.6
1,20230606,True,1,"재택근무를 보장해줌, 성장가능, 휴가 눈치주지 않음, 근무시간 비교적 자유로움, 동...","가끔 업무강도가 높음, 재택의 영향으로 직원들 얼굴 볼 기회가 적음, 성과 내기가 어려움",웹개발,5.0
2,20230527,False,1,눈치볼거 없이 퇴근이 가능하다 그리고 자기개발 하기가 쉽다.,업무량이 많아서 주말에 일을 할 경우가 있다 그런 거 말고는 딱히 없다,B2B영업/기술영업,4.2
3,20230518,False,0,연봉이 좋고 커리어에 도움 할일을 마치면 비교적 자유롭게 퇴근 가능,"젊꼰이 많다 높은 업무강도 책임감등이 필요하다,,,,,",광고/시각디자인,3.4
4,20230517,True,1,자기업무만 마쳤으면 칼퇴가 가능하다. 장기휴가를 가도 크게 눈치보이지 않는다.,업무강도가 높고 항상 일이 많다. 다들 열심히 해서 나도 잘해야할 것 같은 압박이 있다.,기획/전략/경영,4.0
5,20230517,False,1,분위기가 좋고 워라밸가능 업무강도적당 적당한 연봉이다,특정기간에는 업무강도가 있고 야근도 함 그만큼 수당챙겨줌,마케팅/PR/분석,5.0
6,20230516,False,1,칼퇴 해도 아무도 뭐라고 안 하셨고 가끔씩 지각해도 눈치 주시는 거 없어서 좋았습니다,업무강도가 그렇게 높은건 아니였지만 일이 많이 힘들어서 힘들었어요,기획/전략/경영,4.2
7,20230515,True,1,"유연근무제, 재택이 확실히 이점인 회사! 시설도 매우좋아요",성과가 중요하고 꾸준히 성장해야해서 약간의 부담이 있기도합니다,디자인기타,4.4
8,20230515,True,1,식사가 일단 내가 먹고 싶은 종류별 먹을 수 있고 유연근무제가 있어서 좋았다,명확한 지침이 필요하다 업무를 추진할 때 실무자들 위주로 흘러가는 경향이 있다,기획/전략/경영,2.8
9,20230515,True,1,유연근무제+재택근무 가능한 점이 가장 큰 장점입니다 또 네임벨류가 it 대기업 중엔...,"조직문화는 좋은 편이지만, 대기업의 특성상 탑다운은 피할 수 없는 것 같아요ㅠ",웹기획/PM/웹마케팅,4.0


In [40]:
RV_Catch('카카오')

,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,20230728,True,1,사람들 전체적으로 괜찮고 휴가 사용같은게 자유로움 스트레스가 비교적 적은 환경,경영진의 문제가 심각 일단 사이즈는 큰데 앞으로 뭐먹고..?,웹기획/PM/웹마케팅,3.8
1,20230609,True,1,칼퇴 가능 복지 좋음 자유로운 분위기 휴가 이용 자유로움,이쪽 특이긴 하지만 연봉 인상률이 낮음 자유로운 분위기가 독이 되는게 본인이 자율적...,응용프로그램개발,3.6
2,20230602,True,1,연차도 자유롭게 사용 가능하고 쾌적한 근무환경과 좋은 사내 분위기로 만족함.,진입장벽이 있고 업계 전반적으로 성장가능성이 높음. 연봉인상률 양호함. 부서에 따라...,빅데이터/AI,4.6
3,20230526,False,1,자유로운 근무가 가능하다 그리고 워라벨 연봉이 좋다 퇴근 빠르다,업무강도가 있는편이고 개인적인 업무 편향으로 먼가 혼자 일한다,마케팅/PR/분석,5.0
4,20230525,True,1,"유연근무제, 휴가 사용 결재없음 등 자유롭고 복지가 무엇보다 너무 좋습니다",업무에 체계가 없어서 지시도 명확하지 않고..내가 무슨일을 하고 있는거지 하는 확신...,인사/노무/교육,3.6
5,20230516,False,1,판교역이랑 붙어있어 아주 좋다 회사도 크고 실력만 있으면 누군나 입사 가능,실력이 좋아야 입사를 하지만 결국 승진을 위해 학벌이 매우 중요할 것이다,게임,4.6
6,20230515,False,1,장점은 솔직히 다른곳보다는 안정적이고 유대관계나 상하관계가 좋다 라고 생각이든다 배...,단점은 시스템이 변경이 잦아 그 부분에서 따라가는걸 벅차 하는 사람도 있고 그 중 ...,웹개발,4.2
7,20230515,True,1,눈치 볼 필요 없이 칼퇴가 가능하다. 나의 성장을 최대한 지원해 준다,근무강도와 근무 시간이 많은 편이다. 야근도 꽤 자주하는듯하다,재무/세무/IR,5.0
8,20230218,True,1,경력직 기반으로 서로 존중하는 문화. 사내 오아시스 라는 스낵바에 각종 간식과 음료...,형편없는 경영진. 탑다운식 의사결정. 관리자들의 수준이 낮음.,영업관리/지원/영업기획,3.4
9,20221215,True,0,시설이 쾌적하고 편리하며 직장 내 분위기와 환경이 편하다,"사건이 터지면 업무시간과 근무량, 야근이 늘어나며 수당이 비정기적이다",고객관리/인바운드,3.8


In [41]:
from new_catch import RV_Catch

In [42]:
RV_Catch('NC소프트')

,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,20230530,False,1,일 성과만큼 인정은 받을 수 있는 곳. 복지도 거의 다 있어서 좋다,월급에 비해 120프로 일 시키는 곳.. 눈치 많이 봐야하고 정치질도 있지..,요리/영양/제과제빵/바리스타,3.4
1,20230528,True,0,"편하게다니고, 연차많이주고 경조사비를 많이준다, 육아휴직이쉽다","가족회사라 한명에 의해서 쉽게 의사결정바뀌고, 쓸데없는 임원이많다",일반영업,2.8
2,20230518,True,1,"주 20시간인데도 괜찮고 토, 일, 공휴일 쉬어서 좋습니다",애들 때문에 일하면서 화가 나고 스트레스받아서 너무 지칩니다,AS/서비스/수리,4.6
3,20230517,False,0,업무시간이 종료가 되면 바로 퇴근 할 수 있습니다 연차를 원하는 날에 쓸 수 있지만...,서비스업에 근무하였는데 매출이 안나온다고 직원을 구해주지않아 출근하면 연장근무는 3...,외식/식음료,3.2
4,20230516,True,0,"영양사 조리사는 진리의 점by점 그나마 좋은점은 명절에 그나마 돈 주공,, 경조사 ...",바쁜 점포 걸리면 워라벨 저리 가시오 이상한 팀장 만나면 손익 노이로제 걸림 구닥다...,고객관리/인바운드,3.0
5,20230417,True,0,워라벨이 좋다. 눈치 보지 않고 칼퇴가 가능하다. 사옥이 새 것이라 깨끗하다.,성장하는 느낌이 전혀 들지 않음. 고일대로 고인 회사. 발전하려고 하지 않는다.,사무/총무/법무,1.2
6,20230329,False,0,밤 늦은 시각까지의 야근은 없다. 밥 맛있다. 회사 깨끗하고 좋고 마당도 넓다. 게...,대책없는 경영진 노답. 오너 리스크. 연봉이 너무 낮으며 인센티브가 너무 적은편으로...,인사/노무/교육,3.6
7,20230326,True,0,야근이 별로 없고 연차사용에 자유롭습니다. 근데 이건 부바부 사바사라...,경영진이 도대체 뭘 원하는지 모르겠습니다. 점점 퇴보하고 있습니다,일반영업,3.2
8,20221229,False,0,사무직 기준 적당한 워라벨. 급여자체는 낫배드. 마곡 본사의 경우 근무시설 양호. ...,주주 눈치보는 경영진. 주주가 최악임. 성과급은 없다고 보면됨.,재무/세무/IR,3.4
9,20221010,True,1,일잘하는 사람이 많이 퇴사하였지만 그럼에도 불구하고 여전히 잘하는 사람들이 남아있어...,"경영진 능력부족, 일잘하는 사람만 계속시키는 불합리한 구조, 성과급은 개인의 능력이...",요리/영양/제과제빵/바리스타,3.2
